# Laboratório 4
Material desenvolvido por Henrique Margotte e Aurora Pozo para a disciplina de Aprendizado de Máquina, para o curso de Informática Biomédica da UFPR, semestre 2025/2. Códigos baseados em exemplos da biblioteca scikit-learn e nos notebooks Python do livro "Inteligência Artificial: Uma Abordagem de Aprendizado de Máquina", 2ª edição.

# Exemplo 1: Processamento de Linguagem Natural (PLN)
Nos últimos laboratórios, tratamos de diversas técnicas de pré-processamento de dados e aprendemos a como transformar atributos categóricos em formato textual para numérico, mas, como fazemos se os nossos dados são compostos de textos? Vamos pensar sobre isso observando o exemplo a seguir:

In [13]:
# Preparando o ambiente
import pandas as pd
import numpy as np

# Importando base de dados
df = pd.read_csv('title_reviews.csv')
df = df.head(1000) # usando apenas 1000 amostras para exemplo
df.head()

,class,review_title
0,1,Adolescent theology
1,2,A Great Guide for the Mushroom Lover
2,1,"Great book, horrible movie"
3,2,The BEST all-around cookbook I have ever had!
4,2,"Daft, Daffy, but Deft"


Os dados acima são uma amostra da base "Amazon Review Polaridy Dataset", disponibilizada por Xiang Zhang (https://drive.google.com/file/d/0Bz8a_Dbh9QhbaW12WVVZS2drcnM/view?usp=drive_link&resourcekey=0-NlNEsGjcAEPJBwDJtuuZ3Q). Ela é composta de avaliações reais de produtos da Amazon, com a classe sendo atribuída como 1: avaliações negativas; e 2: avaliações positivas. A base ainda possui uma coluna com o título da avaliação e outra com o texto completo, mas que foi suprimida para esse exemplo.

Levando isso em conta, ela não é tão diferente das outras bases que utilizamos, possui uma classe definida e atributos que podem ser utilizados para classificar os dados, o que poderia ser ensinado a um modelo de Aprendizado de Máquina (AM). Mas, como podemos transformar esse texto em informação numérica para um modelo aprender?

Esse tipo de problema faz parte da área de Processamento de Linguagem Natural (PNL, ou NLP, do inglês Natural Language Processing). O PNL engloba diversos problemas relacionados com linguagem natural, ou seja, humana, desde textos escritos, até falas e língua de sinais. No Laboratório de hoje iremos explorar apenas uma amostra do PNL, mas a área é muito mais extensa do que podemos tratar aqui, incluindo sistemas de tradução automática, reconhecimento de fala, os grandes modelos de linguagem (LLMs), entre outros!

## Normalizando os dados
Uma das etapas de pré-processamento que podemos aplicar em textos, antes mesmo de transformá-los para conceitos numéricos, é a normalização. Note que essa normalização não é a mesma que aplicamos nos dados até agora, aqui, o objetivo é deixar o texto um pouco mais compreensível e simples para podermos extrair informações.

Um dos métodos mais simples de fazer isso, é mantendo todas as letras como minúsculas ou maiúsculas, assim o modelo poderá entender que "Exemplo" e "exemplo" são a mesma palavra. Podemos usar os métodos do tipo `str` (string) para isso, com a função `lower` transformando as strings para letras minúsculas e a função `strip` removendo, por padrão, espaços em branco extras, tabulações e quebras de linhas, que podem ser úteis dependendo das técnicas a serem utilizadas.

In [15]:
# normalize the text
df['review_title'] = df['review_title'].str.lower()
df['review_title'] = df['review_title'].str.strip()
df.head()

,class,review_title
0,1,adolescent theology
1,2,a great guide for the mushroom lover
2,1,"great book, horrible movie"
3,2,the best all-around cookbook i have ever had!
4,2,"daft, daffy, but deft"


Isso ainda pode gerar conflitos, como por exemplo "rs", que normalemente se refere a risos, e "RS", sigla do Rio Grande do Sul, seriam tratados como a mesma palavra. Mas, mesmo em palavras sem essa diferença isso pode ocorrer, como na palavra "manga", que pode ser tanto uma fruta quanto uma parte de uma roupa, e existem métodos de contornar isso, como os mecanismos de atenção utilizados por LLMs, e que não abordaremos aqui.

Ainda há outros problemas que poderiam ser tratados nessa etapa, como aproximar palavras equivalentes, como "programador" e "programadora" em contextos em que o gênero não é relevante, ou conjugações de verbos, já que "correr", "corri", "correu" e "corremos" podem ter significados bem próximos, mas que serão tratados diferentemente a depender das técnicas. A tokenização é uma estratégia para contornar esses casos, por exemplo.

Por enquanto, manteremos nossa base de dados do jeito que está, apenas aplicando as funções `lower` e `strip`.

## Removendo palavras irrelevantes
Assim como removemos atributos irrelevantes para o problema nos laboratórios anteriores, podemos excluir palavras que não agregam para o problema que estamos tentando resolver. Por exemplo, os conectivos, preposições e artigos ("e", "com", "as", "o", "de", "para", etc.) podem não ser relevantes. Esses tipos de palavras são chamadas de *stopwords*, e algumas bibliotecas já incluem dicionários e funções próprias para removê-las do texto. Também podem ser removidas palavras manualmente, adicionando-as à lista de *stopwords*, para casos em que uma palavra seja irrelevante em seu contexto, como a palavra "livro" em uma base de dados de avaliações de uma biblioteca.

In [16]:
# remove stop words
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

df['review_title'].apply(lambda x: ' '.join([word for word in x.split() if word not in ENGLISH_STOP_WORDS])).head()

0              adolescent theology
1       great guide mushroom lover
2       great book, horrible movie
3    best all-around cookbook had!
4                daft, daffy, deft
Name: review_title, dtype: object